In [ ]:
import textract
import docx2txt
import nltk
import nltk.corpus
import os
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import shutil
import contractions
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [ ]:
#function from git to clean up expressions
def unicodetoascii(text):

    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")

                 )
    return TEXT

In [ ]:
#tokenize word
nltk.download('punkt')
text = textract.process(r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Argo Archive Word\converted word documents\1-28-1949 Leela Menon(scan)pdf_00001.docx")
text = str(text)
text = unicodetoascii(text.replace("\\n", "\n")) #clean up the doc
print(text)

In [ ]:
#sort by year
nltk.download("stopwords")

tokenized_word = word_tokenize(text)
year = re.findall(r'.*([1-3][0-9]{3})', text)

sorted_year = dict()
file_length = []
directory = r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Argo Archive Text"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    with open(f, encoding= "utf8") as text_file:
        file_str = text_file.read()
        years = re.findall(r'.*([1][89][890123456][0-9])', file_str)
    for year in years:
        if int(year) > 1889:
            if year not in sorted_year.keys():
                this_list = [str(f)]
                sorted_year[year] = this_list
            else:
                sorted_year[year].append(str(f))

#print(sorted_year)



directory = r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Argo Archive Text"
for filename in os.listdir(directory):
    if "copy" in filename:
        f = os.path.join(directory, filename)
        os.remove(f)

#sort files in to separate directories using python
mother_dir = r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\newspaper text in sorted years"
for i in sorted_year.keys():
    newpath = i
    newfolder = os.path.join(mother_dir, i)
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)

In [ ]:
mother_dir = r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Newspaper text in sorted years"

for i in sorted_year.keys():
    parent_dir = mother_dir + "\\" + i
    #print(parent_dir)
    for src_file_path in sorted_year[i]:
        #print(src_file_path)
        dst_file_path = parent_dir + "\\" + src_file_path.split("\\")[-1].split(".")[0] + "(copy).txt"
        #print(dst_file_path)
        shutil.copy(src_file_path, dst_file_path)

In [ ]:
nltk.download("stopwords")

In [ ]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [ ]:
#word frequency

#stopwords = set (stopwords.words("english"))

with open(r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Newspaper text in sorted years\1963\2-21-1963 Leela Menon(scan)_00001(copy).txt", encoding="utf8") as file:
  text = file.read()
  text = " ".join(text.split())
  print(summarize(text, 0.05))
  




In [ ]:
#CHATGPT ......
import transformers

# Load the Bart model and tokenizer
model = transformers.BartModel.from_pretrained('bart-large-cnn')
tokenizer = transformers.BartTokenizer.from_pretrained('bart-large-cnn')

# Encode the text and set the maximum length
with open(r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Newspaper text in sorted years\1963\2-21-1963 Leela Menon(scan)_00001(copy).txt", encoding= "utf-8") as f:
    input_text = f.read()
input_text = "This is the input text that you want to summarize."
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
max_length = 128

# Generate the summary
summary_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)

# Decode the summary
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)


In [ ]:
import transformers
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
with open(r"C:\Users\Cheng\OneDrive\桌面\WIN History Project\Newspaper text in sorted years\1963\2-21-1963 Leela Menon(scan)_00001(copy).txt", encoding= "utf8") as f:
    this_file = f.read()
this_list = [this_file]
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
max_length = 2000
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(this_list, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text[0])